# 차로변경 유형 도출을 위한 최종 선행차량, 후행차량 정보 추출

# Set up

## Import

In [ ]:
import datetime as dt
import os
import pandas as pd
import numpy as np
import itertools
import warnings

from tqdm import tqdm


## Load Dataset

In [66]:

data_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/38_22-10차 회의 준비/221022_인접차량정보/sample.csv'
### 데이터 들어있는 폴더 : 

save_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/38_22-10차 회의 준비/221022_인접차량정보/sample_result.csv'
### 데이터 저장할 폴더 : 설정하면됨

In [83]:
df = pd.read_csv('D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/38_22-10차 회의 준비/221022_인접차량정보/sample.csv',  encoding='cp949')

## Create mutual columns

In [84]:
df["LV1_HV0_x"] = 0   #43번열
df["LV1_HV0_y"] = 0   #44번열        
df["LV2_HV_x"] = 0   #45번열
df["LV2_HV_y"] = 0   #46번열
df["HV_FV0_x"] = 0   #47번열
df["HV_FV0_y"] = 0   #48번열

In [85]:
df = df.sort_values(['Vehicle ID', 'Frame ID'], ignore_index=True)    

temp1 = pd.DataFrame()

In [86]:
veh_list = df['Vehicle ID'].unique()

In [72]:
#df.columns.get_loc('Vehicle ID')

# RUN

In [93]:
warnings.filterwarnings(action = 'ignore') # pandas 경고메시지 숨기기

temp_total = pd.DataFrame()

for i in tqdm(veh_list): # i = 차량별. Vehicle ID별로
            
    temp = df[df['Vehicle ID'] == i] # 각 차량별 궤적 자료
            
    LV1_frm = temp['LV1_ID'] # 각 차량의 프레임별 현재차로 선행차량 리스트
    FV_frm = temp['FV_ID'] # 각 차량의 프레임별 목표차로 후행차량
            
    c = 0
    d = 0
            
    if len(LV1_frm) >= 1: #현재차로 선행차량이 있으면
        c = len(LV1_frm) - 1 # 현재차로 선행차량 마지막 프레임 행번호
            
    if len(FV_frm) >= 1: #목표차로 후행차량이 있으면
        d = len(FV_frm) - 1 # 목표차로 후행차량 마지막 프레임 행번호

    print(f'Vehicle ID = {i}, c = {c}, d = {d}')
    
    while LV1_frm.iloc[c] and (LV1_frm.iloc[c] == 0 or pd.isnull(LV1_frm.iloc[c]) == True): # LV1_frm[c] : 현재차로 선행차량 번호가 0이 되거나, Nan인 경우
                
        if c >= 1:
            c -= 1 # c가 1보다 크면 1을 빼준다. # 
                
        else:  
            break
        print(f'While 최종 c = {c}')

    while FV_frm.iloc[d] and (FV_frm.iloc[d] == 0 or pd.isnull(FV_frm.iloc[d]) == True):
                
        if d >= 1:
            d -= 1
                
        else:  
            break
        print(f'While 최종 d = {d}')
            
    ## 지금까지의 output : 각 Vehicle ID별 최종 FV, 최종 LV1이 나오는것
    print(f'Vehicle ID = {i}, LV1 = {LV1_frm.iloc[c]}, FV = {FV_frm.iloc[d]}')
    
    ###########################################################################################
    
    e = 0
    f = 0

    for k in range(0, len(temp)):

        ############ LV2 찍기 ##################

        if temp['LV2_ID'].iloc[k] > 0: # LV2_ID의 K번째 행 값이 0 이상이면 = LV2가 존재하면
                    
            temp['LV2_HV_x'].iloc[k] = abs(temp['LV2_x'].iloc[k] - temp['Local X (m)'].iloc[k])
            temp['LV2_HV_y'].iloc[k] = abs(temp['LV2_y'].iloc[k] - temp['Local Y(m)'].iloc[k])
                    
        elif temp['LV2_ID'].iloc[k] == -1:
                    
            if temp['Local X (m)'].iloc[k] > temp['LV2_x'].iloc[k]:
                temp['LV2_HV_x'].iloc[k] = temp['Local X (m)'].iloc[k] - temp['LV2_x'].iloc[k]
                temp['LV2_HV_y'].iloc[k] = abs(temp['Local Y(m)'].iloc[k] - temp['LV2_HV_y'].iloc[k])
                            
            else:
                pass
                    
#             else: # 남양주TG 서울방향이 아니면
#                 if temp['LV2_x'].iloc[k] > temp['Local X (m)'].iloc[k]:
#                     temp['LV2_HV_x'].iloc[k] = temp['LV2_x'].iloc[k] - temp['Local X (m)'].iloc[k]
#                     temp['LV2_HV_y'].iloc[k] = abs(temp['Local Y(m)'].iloc[k] - temp['LV2_y'].iloc[k])
#                 else:
#                     continue
        else:
            pass
            
            
        ############ LV1 찍기 ##################

        if len(LV1_frm) > 0: # 현재차로 선행차량이 있으면
                    
            if e == 0:
                
                if temp['LV1_ID'].iloc[k] == LV1_frm.iloc[c]: # LV1 번호
                            
                    e += 1
                    
                    temp['LV1_HV0_x'].iloc[k] = abs(temp['LV1_x'].iloc[k] - temp['Local X (m)'].iloc[k])
                    temp['LV1_HV0_y'].iloc[k] = abs(temp['LV1_y'].iloc[k] - temp['Local Y(m)'].iloc[k])
                    
                else:
                    pass

            elif k >= 1:
                temp['LV1_HV0_x'].iloc[k] = temp['LV1_HV0_x'].iloc[k-1]
                temp['LV1_HV0_y'].iloc[k] = temp['LV1_HV0_y'].iloc[k-1]

            else:
                pass
                
        else:
            pass

        
        ############ FV 찍기 ##################
        
        if len(FV_frm) > 0:
                    
            if f == 0:

                if temp['FV_ID'].iloc[k] == FV_frm.iloc[d]:   # FV 번호
                            
                    f += 1

                    temp['HV_FV0_x'].iloc[k] = abs(temp['Local X (m)'].iloc[k] - temp['FV_x'].iloc[k])
                    temp['HV_FV0_y'].iloc[k] = abs(temp['Local Y(m)'].iloc[k] - temp['FV_y'].iloc[k])

            elif k >= 1:
                temp['HV_FV0_x'].iloc[k] = temp['HV_FV0_x'].iloc[k-1]
                temp['HV_FV0_y'].iloc[k] = temp['HV_FV0_y'].iloc[k-1]

            else:
                pass

    try:
        temp.drop(['Unnamed: 0'], axis = 1, inplace = True)

    except:
        continue
        
    temp_total = pd.concat([temp_total, temp])

temp_total.to_csv(save_dir, encoding='cp949')

 24%|███████████████████▊                                                               | 5/21 [00:00<00:00, 44.54it/s]

Vehicle ID = 6, c = 15, d = 15
While 최종 d = 14
While 최종 d = 13
While 최종 d = 12
While 최종 d = 11
While 최종 d = 10
While 최종 d = 9
While 최종 d = 8
While 최종 d = 7
While 최종 d = 6
While 최종 d = 5
While 최종 d = 4
While 최종 d = 3
While 최종 d = 2
While 최종 d = 1
While 최종 d = 0
Vehicle ID = 6, LV1 = 3, FV = nan
Vehicle ID = 173, c = 8, d = 8
Vehicle ID = 173, LV1 = -1, FV = 172.0
Vehicle ID = 540, c = 6, d = 6
While 최종 d = 5
While 최종 d = 4
While 최종 d = 3
While 최종 d = 2
While 최종 d = 1
While 최종 d = 0
Vehicle ID = 540, LV1 = -1, FV = nan
Vehicle ID = 871, c = 3, d = 3
Vehicle ID = 871, LV1 = -1, FV = 873.0
Vehicle ID = 873, c = 13, d = 13
While 최종 d = 12
While 최종 d = 11
While 최종 d = 10
While 최종 d = 9
While 최종 d = 8
While 최종 d = 7
While 최종 d = 6
While 최종 d = 5
While 최종 d = 4
While 최종 d = 3
While 최종 d = 2
While 최종 d = 1
While 최종 d = 0
Vehicle ID = 873, LV1 = 871, FV = nan
Vehicle ID = 929, c = 4, d = 4
While 최종 d = 3
While 최종 d = 2
While 최종 d = 1
While 최종 d = 0
Vehicle ID = 929, LV1 = -1, FV = nan
Vehicle ID

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 54.65it/s]

Vehicle ID = 10181, c = 6, d = 6
While 최종 d = 5
While 최종 d = 4
While 최종 d = 3
While 최종 d = 2
While 최종 d = 1
While 최종 d = 0
Vehicle ID = 10181, LV1 = 10178, FV = nan
Vehicle ID = 10190, c = 4, d = 4
Vehicle ID = 10190, LV1 = -1, FV = 10193.0
Vehicle ID = 10321, c = 3, d = 3
While 최종 d = 2
While 최종 d = 1
While 최종 d = 0
Vehicle ID = 10321, LV1 = -1, FV = nan
Vehicle ID = 10370, c = 2, d = 2
While 최종 d = 1
While 최종 d = 0
Vehicle ID = 10370, LV1 = -1, FV = nan
Vehicle ID = 10596, c = 9, d = 9
Vehicle ID = 10596, LV1 = -1, FV = 10605.0
Vehicle ID = 10632, c = 13, d = 13
Vehicle ID = 10632, LV1 = -1, FV = 10639.0
Vehicle ID = 10702, c = 3, d = 3
Vehicle ID = 10702, LV1 = -1, FV = 10729.0
Vehicle ID = 20193, c = 9, d = 9
While 최종 d = 8
While 최종 d = 7
While 최종 d = 6
While 최종 d = 5
While 최종 d = 4
While 최종 d = 3
While 최종 d = 2
While 최종 d = 1
While 최종 d = 0
Vehicle ID = 20193, LV1 = 20191, FV = nan
Vehicle ID = 20240, c = 10, d = 10
Vehicle ID = 20240, LV1 = -1, FV = 20236.0
Vehicle ID = 20250, c 

In [64]:
df.to_csv('D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/38_22-10차 회의 준비/221022_인접차량정보/columns.csv',  encoding='cp949')